In [ ]:
import pandas as pd
import geopandas as gpd
import folium

In [ ]:
transport = pd.read_csv('data_files/transport_data.csv')
wards = gpd.read_file('data_files/NI_Wards.shp')

merged = wards.merge(transport, left_on='Ward Code', right_on='Ward Code')

df = pd.read_csv('data_files/Airports.csv') # read the csv data
# create a new geodataframe
airports = gpd.GeoDataFrame(df[['name', 'website']], # use the csv data, but only the name/website columns
                            geometry=gpd.points_from_xy(df['lon'], df['lat']), # set the geometry using points_from_xy
                            crs='epsg:4326') # set the CRS using a text representation of the EPSG code for WGS84 lat/lon


merged['NumBus_ratio'] = round ( merged['Population'] / merged['NumBus'] ) #creation of new column with the number of people per bus stop in the ward
m = merged.explore('NumBus_ratio', # show the NumBus column, refered to the number of people per bus stops in the ward
                   cmap='plasma', # use the 'plasma' colormap from matplotlib
                   legend_kwds={'caption': 'Number of people per bus stops in the ward'} # set the caption to a longer explanation
                  )

airport_args = {
    'm': m, # add the markers to the same map we just created
    'marker_type': 'marker', # use a marker for the points, instead of a circle
    'popup': True, # show the information as a popup when we click on the marker
    'legend': False, # don't show a separate legend for the point layer
    'marker_kwds': {'icon': folium.Icon(color='red', icon='plane', prefix='fa')} # make the markers red with a plane icon from FA
}

# use the airport_args with the ** unpacking operator - more on this next week!
airports.explore('name', **airport_args)

m

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ni_utm = ccrs.UTM(29)  # create a Universal Transverse Mercator reference system to transform our data.
# NI is in UTM Zone 29, so we pass 29 to ccrs.UTM()
ax = plt.axes(projection=ni_utm)  # create an axes object in the figure, using a UTM projection,
# where we can actually plot our data.



xmin, ymin, xmax, ymax = outline.total_bounds
# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm)  # because total_bounds
# gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

In [ ]:
NumBus_ratio = round( merged['Population'] / merged['NumBus'] )
print(NumBus_ratio)

In [ ]:
df = pd.read_csv('data_files/Bus_rail_stations.csv') # read the csv data

# create a new geodataframe
Bus_rail_stations = gpd.GeoDataFrame(df[['Station', 'ID', 'Type', 'Type~Def']], # use the csv data, but only the name/website columns
                            geometry=gpd.points_from_xy(df['Easting'], df['Northing']), # set the geometry using points_from_xy
                            crs='epsg:4326') # set the CRS using a text representation of the EPSG code for WGS84 lat/lon

Bus_rail_stations.head(5) # show the new geodataframe

In [ ]:
marker_icons = {
    'B': 'bus',
    'R': 'train',
    'I': 'station'
}

# Function to add marker with specified icon based on 'Type' column
def add_marker(row):
    icon = marker_icons.get(str(row['Type']), 'question')  # Default icon type if Type is not A, B, or C
    folium.Marker(location=[row.geometry.y, row.geometry.x], icon=folium.Icon(icon=icon)).add_to(m)

Bus_rail_stations.explore(lambda x: add_marker(x))

m

In [ ]:
# Define marker properties
Bus_rail_stations_args = {
    'marker_type': 'marker',  # use a marker for the points, instead of a circle
    'popup': True,  # show the information as a popup when we click on the marker
    'legend': False,  # don't show a separate legend for the point layer
    'marker_kwds': {'prefix': 'fa'}  # make the markers red with a plane icon from FA
}

for index, row in Bus_rail_stations.iterrows():
    location = [row.geometry.y, row.geometry.x]  # Coordinates are stored in a 'geometry' column
    if row.Type == 'R':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='red', icon='train')
    elif row.Type == 'B':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='blue', icon='bus')
    elif row.Type == 'I':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='orange', icon='car')
    else:
        continue
    
    marker = folium.Marker(location=location, **Bus_rail_stations_args)
    marker.add_to(m)    #may be it is posible to change by geodataframe.explore(lambda x: add_marker(x))


# Display the map
m

In [ ]:
# Define marker properties
Bus_rail_stations_args = {
    'marker_type': 'marker',  # use a marker for the points, instead of a circle
    'popup': True,  # show the information as a popup when we click on the marker
    'legend': False,  # don't show a separate legend for the point layer
    'marker_kwds': {'prefix': 'fa'}  # make the markers red with a plane icon from FA
}

for index, row in Bus_rail_stations.iterrows():
    #location = [row.geometry.y, row.geometry.x]  # Coordinates are stored in a 'geometry' column
    if row.Type == 'R':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='red', icon='train')
    elif row.Type == 'B':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='blue', icon='bus')
    elif row.Type == 'I':
        Bus_rail_stations_args['marker_kwds']['icon'] = folium.Icon(color='orange', icon='bus')
    else:
        continue
    
    marker = folium.Marker(location=[row.geometry.y, row.geometry.x], **Bus_rail_stations_args)
    marker.add_to(m)    #may be it is posible to change by geodataframe.explore(lambda x: add_marker(x))


# Display the map
m